In [27]:
import aacgmv2
import pandas as pd
import numpy as np
from multiprocessing import Pool
import math, os, shutil
from tqdm.auto import tqdm
import glob
import datetime
from scipy import signal

from spacepy.coordinates import Coords
from spacepy.time import Ticktock
import fnmatch
import sys
from scipy.interpolate import LinearNDInterpolator, interp1d, griddata

%matplotlib inline

## Settings

In [4]:
dtime_storm_start = datetime.datetime(2011,5,21,13,40) 

dtime_sim_start = datetime.datetime(2011,5,20)

t_step_minutes = 5 # minutes


plot_start_delta  = 3  # hours before storm onset to start making plots. set to -1 to run the whole time
plot_end_delta    = 9  # hours after storm onset to end plots. Set to -1 to run for the whole time

sami_data_path = "/home/axb170054/scratch/GITM-testing/test_folders/step_function_driving/SAMI3-stretch/"


global_lat_lim = 65 # will limit SAMI output data to this GEO latitude


sample_rate_min = 5 # SHOULD be the same as t_step_minutes above. If not, something will be wrong. Won't throw errors. Gotta pay attention.
low_cut = 100 # min, lowest freq wave the filter will allow thru
high_cut = 40 # min, highest freq the filter will allow thru


 #Do we want to run the filtering? if not, this will need to be done later.
to_filter = False # This is not implemented yet!!! Would be a good project for someone.
debug = False # make plots at the end? I would leave this as False. At the bottom there are some simple plotting routines. 


In [5]:
cols = ['edens', 'hplusdens', 'oplusdens', 'noplusdens', 'o2plusdens', 'heplusdens', 'n2plusdens', 'nplusdens', 'hdens', 'odens', 'nodens', 'o2dens', 'hedens', 'n2dens', 'ndens']

# above is all cols (that I care about), below is just edens. If we want to output more columns, add them to data_files below too.

# cols = ['edens']

Available columns (for now) are:

['edens', 'hplusdens', 'oplusdens', 'noplusdens', 'o2plusdens', 'heplusdens', 'n2plusdens', 'nplusdens', 'hdens', 'odens', 'nodens', 'o2dens', 'hedens', 'n2dens', 'ndens']

In [6]:
#Data mapped to a grid will go here
data_out_path = '/home/axb170054/scratch/pickles/SimStormPaper/simstorm_sami_files/'
save_raw = False # output txt files for raw data

In [7]:
#Options for multithreading. 
thread = True 
num_workers = len(cols) # adjust this to fit your system.


### Speficy the output grid

Lats & lons are calculated, altitudes must be specified.

In [8]:
out_grid_lats = 65 #number of lats
out_grid_lons = 75 #number of lons

In [9]:

out_lats = np.linspace(-global_lat_lim,global_lat_lim, out_grid_lats)
out_lons = np.linspace(0,360, out_grid_lons +1)[1:] # we don't need both 0 & 360. the 1's deal with that.
out_alts = np.array([250,300,350,400,450,500,550,600,700,800,840,880,900,1000]) 

In [10]:
out_lats, out_lons, out_alts, len(out_lats), len(out_lons), len(out_alts)

(array([-65.     , -62.96875, -60.9375 , -58.90625, -56.875  , -54.84375,
        -52.8125 , -50.78125, -48.75   , -46.71875, -44.6875 , -42.65625,
        -40.625  , -38.59375, -36.5625 , -34.53125, -32.5    , -30.46875,
        -28.4375 , -26.40625, -24.375  , -22.34375, -20.3125 , -18.28125,
        -16.25   , -14.21875, -12.1875 , -10.15625,  -8.125  ,  -6.09375,
         -4.0625 ,  -2.03125,   0.     ,   2.03125,   4.0625 ,   6.09375,
          8.125  ,  10.15625,  12.1875 ,  14.21875,  16.25   ,  18.28125,
         20.3125 ,  22.34375,  24.375  ,  26.40625,  28.4375 ,  30.46875,
         32.5    ,  34.53125,  36.5625 ,  38.59375,  40.625  ,  42.65625,
         44.6875 ,  46.71875,  48.75   ,  50.78125,  52.8125 ,  54.84375,
         56.875  ,  58.90625,  60.9375 ,  62.96875,  65.     ]),
 array([  4.8,   9.6,  14.4,  19.2,  24. ,  28.8,  33.6,  38.4,  43.2,
         48. ,  52.8,  57.6,  62.4,  67.2,  72. ,  76.8,  81.6,  86.4,
         91.2,  96. , 100.8, 105.6, 110.4, 115.2, 120

## Constants:

In [11]:
geo_grid_files = {'glat':'glatu.dat','glon':'glonu.dat','alt':'zaltu.dat', 
                  'mlat':'blatu.dat','mlon':'blonu.dat','malt':'baltu.dat'}


data_files = {'edens':'deneu.dat', 'hplusdens':'deni1u.dat','oplusdens':'deni2u.dat',
              'noplusdens':'deni3u.dat', 'o2plusdens':'deni4u.dat',
              'heplusdens':'deni5u.dat', 'n2plusdens':'deni6u.dat', 
              'nplusdens':'deni7u.dat','hdens':'denn1u.dat','odens':'denn2u.dat', 
              'nodens':'denn3u.dat', 'o2dens':'denn4u.dat', 'hedens':'denn5u.dat', 
              'n2dens':'denn6u.dat', 'ndens':'denn7u.dat'}

time_file = 'time.dat'

## Define Functions

In [12]:
def get_grid_elems_from_parammod(data_dir):
    """
    Will look for: words = ['nz0','nf','nl'] in SAMI files. You can either run this or set the elements yourself.
    
    inputs:
    ------
    sami path
    
    outputs:
    -------
    nz,nf,nlt,nt :
    - nz  = num points along field line
    - nf  = num field lines along each mag lon
    - nlt = num mag lons
    - nt  = num times
    
    """
    
    #Make sure that we only grab the first instance of each var in the file. 
        # SOmetimes they repeat and we don't want them
    returns = [False, False, [False, False], False]
    
    with open(data_dir  + 'parameter_mod.f90', 'r') as fp:
    # read all lines in a list
        lines = fp.readlines()
        for line in lines:
            # check if string present on a current line
            
            if not returns[0]:
                if line.find('nz0') != -1:
                    nz0 = []
                    for l in line:
                        if l.isdigit():
                            nz0.append(l)
                    if len(nz0[1:4]) == 3:
                        nz = int(''.join(nz0[1:4]))
                        returns[0] = True
            
            if not returns[1]:
                if line.find('nf') != -1:
                    nf = []
                    for l in line:
                        if l.isdigit():
                            nf.append(l)
                    nf = int(''.join(nf))
                    returns[1] = True
                    
            if not returns[2][0]:
                if line.find('nl ') != -1:
                    nl = []
                    for l in line:
                        if l.isdigit():
                            nl.append(l)
                    nl = int(''.join(nl))
                    returns[2][0] = True
                    
            if not returns[2][1]:
                if line.find('numwork ') != -1:
                    numwork = []
                    for l in line:
                        if l.isdigit():
                            numwork.append(l)
                    numwork = int(''.join(numwork))
                    returns[2][1] = True
                    
    #time
    with open(data_dir  + 'time.dat', 'r') as fp:
        lines = fp.readlines()
        nt = len(lines) - 1
            
    return nz, nf, numwork*(nl - 2), nt



In [13]:
def make_times(t0, nt, plot_start_delta = None, plot_end_delta = None):
    times = []
    hrs_since_storm_start = []
    
    for t in range(nt):
        time_here = pd.Timestamp(dtime_sim_start) + t * pd.Timedelta(5, 'minutes')
        times.append(time_here.to_pydatetime())
        hrs = (time_here - dtime_storm_start)/pd.Timedelta(1, 'hour')
        hrs_since_storm_start.append(hrs)
        
        
        
    times_df = pd.read_fwf(os.path.join(sami_data_path, 'time.dat'), 
                            names = ['istep', 'hour', 'minute', 'second', 'hrdelta'], infer_nrows=115)
    times_df.pop('istep');

    times_list = []
    for hr in times_df['hrdelta']:
        times_list.append(dtime_sim_start + datetime.timedelta(hours = hr))    
    
    truths = np.array([pd.Timestamp(times_list[t]).round('T') == times[t] for t in range(len(times))])
    if truths.sum() != len(truths):
        raise ValueError('The times are wrong! Somehow this needs to be fixed. probably outputting fake files again. Take a look and debug before proceeding.')
        
    #maybe chop the time lists, depending on if the plot start/end are given.
    # adjusted to allow for -1 in plot start/end deltas (plot all times)
        
    if plot_start_delta and plot_end_delta:
        if plot_start_delta != -1:
            start_idx = np.argmin(np.abs(np.array(times) 
                                     - (dtime_storm_start - pd.Timedelta(plot_start_delta, 'hour'))))
        else:
            start_idx = 0
            
        if plot_end_delta != -1:
            end_idx = np.argmin(np.abs(np.array(times) 
                                     - (dtime_storm_start + pd.Timedelta(plot_end_delta, 'hour'))))
        elif plot_end_delta == -1:
            end_idx = len(times)
        else:
            end_idx = len(times)
        
        times = times[start_idx:end_idx]
        hrs_since_storm_start = hrs_since_storm_start[start_idx:end_idx]
        times_list = times_list[start_idx:end_idx]
        
        return times, hrs_since_storm_start, times_list, (start_idx, end_idx)
        
    elif plot_start_delta != plot_end_delta:
        raise ValueError('You cannot specify one and not the other!')
    
    return times, hrs_since_storm_start, times_list

In [15]:
def get_sami_grid(sami_data_path = sami_data_path, geo_grid_files = geo_grid_files):
    """
    return:
    dictionary
        - grid['key'][nlt , nf, nz]
        
        key = grid file: glat, glon, alt, etc.
        nlt, nf, nz; see get_grid_elems_from_parammod
        
        

    """
    grid = {}

    for f in geo_grid_files:
        file = open(os.path.join(sami_data_path, geo_grid_files[f]), 'rb')
        raw = np.fromfile(file, dtype='float32')[1:-1].copy()
        file.close()

        grid[f] = raw.reshape(nlt,nf,nz).copy()
    return grid

In [28]:
def read_sami_data(cols, nts):
    """
    Read in sami data for the specified columns and return sama data dict
    
    inputs:
    -------
    cols: list-like
        - Columns you want data for. Does not have to be everything
    
    nts: int OR tuple/list
        - either nt (number of times) if you want all sami data from simulation or:
        - nts (start_time, end_time) if you want plots from a select time period.
        
        
    return:
    dictionary
        - sami_data['key'][nlt , nf, nz, ntime]
        
        key = column: edens, hdens, etc.
        nlt, nf, nz; see get_grid_elems_from_parammod
        ntime = time index.
    """
    sami_data = {}
    
    #handle cut time list and full time list
    if type(nts) != int:
        t_start = nts[0]
        t_end   = nts[1]
        ntimes = t_end - t_start
    else:
        t_start = 0
        t_end = nt
        ntimes = nt
        
    pbar = tqdm(total = len(cols) * ntimes, desc = 'reading sami data')

    for f in cols:

        sami_data[f] = np.zeros((nlt,nf,nz,ntimes))

        file = open(os.path.join(sami_data_path, data_files[f]), 'rb')
        for t in range(t_end):
            raw = np.fromfile(file, dtype='float32', count = (nz*nf*nlt)+2)[1:-1]
            if t >= t_start:
                sami_data[f][:,:,:,t-t_start] = raw.reshape(nlt,nf,nz).copy()
                pbar.update(1)
        file.close()
    pbar.close()
        
    return sami_data

In [30]:
## Bandpass filtering


def make_filter(params = None):
    # Define the cutoff frequencies
    lowcut = 1/(100/60)  # 100 minutes in units of sample^-1
    highcut = 1/(30/60) # 30 minutes in units of sample^-1

    # Define the Butterworth filter
    nyquist = 0.5 * 5 # 5 minutes is the sampling frequency
    low = lowcut / nyquist
    high = highcut / nyquist
    sos = signal.butter(2, [low, high], btype='bandstop', output='sos')
    return sos

def remove_background(time_series, sos):


    # Apply the filter to the time series
    filtered_data = signal.sosfiltfilt(sos, time_series)

    return filtered_data



In [18]:
def geo_to_cart(lats_, lons_, alts_, ntime):
    """
    get cartesian from a grid slice
    """
    
    coord_arr = []
    
    if type(alts_) != list:
        if type(alts_) != np.array:
            if type(alts_) != np.ndarray:
                alts_ = [ alts_ ]


    for lat in lats_:
        for lon in lons_:
            for alt in alts_:
                coord_arr.append([(alt + 6371)/6371, lat, lon])
                
    dtime = times[ntime]

    coords = Coords(coord_arr, 'GEO','sph')
    coords.ticks = Ticktock([dtime for k in range(len(coord_arr))])

    newcoords = coords.convert('GEO','car')
    
    return newcoords

In [32]:
def grid_to_cartesian(grid, ntime):
    """ get cartesian coords of the output grid
    
    
    """
    coords0 = list(zip(grid['malt'].flatten()/ 6371 , grid['mlat'].flatten(), grid['mlon'].flatten()))
    dtime = times[ntime]

    coords = Coords(coords0, 'CDMAG','sph')
    coords.ticks = Ticktock([dtime for k in range(len(coords0))])

    newcoords = coords.convert('GEO','car')
    
    return newcoords

## Read in data

In [20]:
nz, nf, nlt, nt = get_grid_elems_from_parammod(sami_data_path)

In [21]:
times, hrs, times_list, nts = make_times(dtime_sim_start, nt, plot_start_delta, plot_end_delta)
new_nt = np.diff(nts)[0]

In [22]:
## OR.... to run for all times:
# times, hrs, times_list = make_times(dtime_sim_start, nt)


In [23]:
print(nlt, nf, nz, nts, new_nt)
nt = new_nt

80 72 256 (416, 560) 144


In [24]:
grid = get_sami_grid()

In [25]:
for g in grid.keys():
    print(g, grid[g].shape)
    

glat (80, 72, 256)
glon (80, 72, 256)
alt (80, 72, 256)
mlat (80, 72, 256)
mlon (80, 72, 256)
malt (80, 72, 256)


In [29]:
# sami grid
sami_data = read_sami_data(cols, nts)
# or
# sami_data = read_sami_data(cols, nt)

print('sami data shape: ', sami_data[cols[0]].shape)

reading sami data:   0%|          | 0/2160 [00:00<?, ?it/s]

sami data shape:  (80, 72, 256, 144)


In [33]:
if to_filter:
    print('Calculating fits. This will take a moment...')
    fits_sami = make_fits(sami_data)

## Some setup

In [27]:
print('example data shape: ', sami_data['edens'].shape)

example data shape:  (80, 72, 256, 96)


## Do the interpolation.


1. Limit the iput data to a reasonable alt range.
2. map inputs & outputs to cartesian (at each datetime)
3. do interpolations
4. output

In [46]:
norm_alts = (grid['alt'].flatten() < (max(out_alts) + 300)) & (grid['alt'].flatten() > (min(out_alts)-75))

In [34]:
out_alts, out_lons

(array([ 250,  300,  350,  400,  450,  500,  550,  600,  700,  800,  840,
         880,  900, 1000]),
 array([  4.8,   9.6,  14.4,  19.2,  24. ,  28.8,  33.6,  38.4,  43.2,
         48. ,  52.8,  57.6,  62.4,  67.2,  72. ,  76.8,  81.6,  86.4,
         91.2,  96. , 100.8, 105.6, 110.4, 115.2, 120. , 124.8, 129.6,
        134.4, 139.2, 144. , 148.8, 153.6, 158.4, 163.2, 168. , 172.8,
        177.6, 182.4, 187.2, 192. , 196.8, 201.6, 206.4, 211.2, 216. ,
        220.8, 225.6, 230.4, 235.2, 240. , 244.8, 249.6, 254.4, 259.2,
        264. , 268.8, 273.6, 278.4, 283.2, 288. , 292.8, 297.6, 302.4,
        307.2, 312. , 316.8, 321.6, 326.4, 331.2, 336. , 340.8, 345.6,
        350.4, 355.2, 360. ]))

In [59]:


pbar = tqdm(total = len(times)*len(cols), desc = 'making fits')

# this will be very messy. clean up after the processing is done.

def interp_grid(col):
    preds = {}
    preds[col] = np.zeros([len(times),len(out_lats), len(out_lons),  len(out_alts)])
    
    for ntime, dt in enumerate(times):

        grid_cart = grid_to_cartesian(grid, ntime)

        xs = grid_cart.data[:,0][norm_alts]
        ys = grid_cart.data[:,1][norm_alts]
        zs = grid_cart.data[:,2][norm_alts]


        loc_grid = list(zip(xs, ys, zs))


        out_grid = geo_to_cart(out_lats, out_lons, out_alts, ntime)

        out_xs = out_grid.data[:,0]
        out_ys = out_grid.data[:,1]
        out_zs = out_grid.data[:,2]

        datas = sami_data[col][:,:,:,ntime].flatten()[norm_alts]


        interp = LinearNDInterpolator(loc_grid , datas, rescale = True)
        pred = interp(list(zip(out_xs, out_ys, out_zs)))

        preds[col][ntime] = pred.reshape([len(out_lats), len(out_lons), len(out_alts)])

        pbar.update(1)

    return preds
        



making fits:   0%|          | 0/1440 [00:00<?, ?it/s]

In [48]:
#Thread the above function, if thread option is set.

if thread:
    with Pool(len(cols)) as pool:

        pred_inter = pool.map(interp_grid, cols)
        
    ## Clean up predictions... Returns a list of dicts when threaded.

    
else:
    pred_inter = []
    for col in cols:
        pred_inter.append(interp_grid[col])
        


/home/axb170054/.conda/envs/py38/lib/python3.8/site-packages/spacepy/coordinates.py:248: DeprecationWarning: No coordinate backend specified; using SpacePy. This default changed from IRBEM in version 0.4.0
  warnings.warn('No coordinate backend specified; using SpacePy.'
/home/axb170054/.conda/envs/py38/lib/python3.8/site-packages/spacepy/coordinates.py:248: DeprecationWarning: No coordinate backend specified; using SpacePy. This default changed from IRBEM in version 0.4.0
  warnings.warn('No coordinate backend specified; using SpacePy.'
/home/axb170054/.conda/envs/py38/lib/python3.8/site-packages/spacepy/coordinates.py:248: DeprecationWarning: No coordinate backend specified; using SpacePy. This default changed from IRBEM in version 0.4.0
  warnings.warn('No coordinate backend specified; using SpacePy.'
/home/axb170054/.conda/envs/py38/lib/python3.8/site-packages/spacepy/coordinates.py:248: DeprecationWarning: No coordinate backend specified; using SpacePy. This default changed from I

14676500489.940033 hedens
18334775737.929016 n2dens
15092640235.64255 hdens
12645056.441270534 nplusdens
49535726.97728915 hplusdens
1427979.812719341 heplusdens
6672758601.606803 ndens
274353.2180117546 n2plusdens
376333801739.8815 odens
1621254917.6399298 oplusdens
14678513.921858165 nodens
337619570.9452592 o2dens
1685539895.07889 edens
302855.96013359167 noplusdens
99007.41826828575 o2plusdens


In [60]:
#Clean the output pred arrays. Things got weird if you had threading on.

preds_cleaned = {}
for p in pred_inter:
    for k in p.keys():
        preds_cleaned[k] = p[k]
preds = preds_cleaned
del preds_cleaned

In [ ]:


if debug:
    a = 5
    
    ## COmpare maps
    plt.imshow(preds[cols[0]][0,:,:,a], origin = 'lower', extent = [min(out_lons), max(out_lons), min(out_lats), max(out_lats)], aspect = 'auto')
    
    plt.show()
    plt.close()
    
    
    
    alt_mask_plot = (np.abs(grid['alt'].flatten() - out_alts[a]) < 10)
    plt.scatter(grid['glon'].flatten()[alt_mask_plot], grid['glat'].flatten()[alt_mask_plot], c = sami_data['edens'][:,:,:,ntime].flatten()[alt_mask_plot])
    plt.ylim(min(out_lats), max(out_lats))

    plt.show()
    plt.close()






In [ ]:
if to_filter and debug:
    sos = make_filter()
    filtered = {}
    for col in cols:
        filtered[col] = signal.sosfiltfilt(sos, preds[col], axis = 0)
        
    if debug:
        plt.imshow(preds[cols[0]][0,:,:,a], origin = 'lower', extent = [min(out_lons), max(out_lons), min(out_lats), max(out_lats)], aspect = 'auto')
        
        plt.show()
        plt.close()
        
        
        plt.imshow(100*(preds[cols[0]][25,:,:,a] - filtered[cols[0]][25,:,:,a])/preds[cols[0]][1,:,:,a], origin = 'lower', 
                   extent = [min(out_lons), max(out_lons), min(out_lats), max(out_lats)], aspect = 'auto', vmin = -5, vmax = 5)
        plt.colorbar()
        plt.show()
        plt.close()
        
        for l in range(0, len(out_lons),4):
            plt.figure(figsize = (8,4))
            plt.imshow((100*(preds[cols[0]][:,:,l,a] - filtere[cols[0]]d[:,:,l,a])/preds[cols[0]][:,:,l,a]).T, 
                       extent = [min(hrs), max(hrs), min(out_lats), max(out_lats)], aspect = 'auto', vmin = -4, vmax = 4)
            plt.colorbar(label = 'edends % over background at ' + str(out_alts[a]) + ' km')
            plt.title('glon = ' + str(out_lons[l].round(2)))
            plt.show()
            plt.close()

## Save everything to files. 

In [63]:
sami_data['edens'].shape

(80, 72, 256, 96)

In [73]:

print('writing files...')

np.array(times).tofile(data_out_path + 'times', format = '%s', sep = ',')
out_lats.tofile(data_out_path + 'out-lats', sep = ',')
out_lons.tofile(data_out_path + 'out-lons', sep = ',')
out_alts.tofile(data_out_path + 'out-alts', sep = ',')

np.array(preds[cols[0]].shape).tofile(data_out_path + 'out-shape', sep = ',')

for col in cols:

    preds[col].tofile(data_out_path + 'preds-' + col, sep = ',')
    
    if to_filter:
        filtered[col].file(data_out_path + 'bp_filtered-' + col, sep = ',')

if save_raw:
    for col in cols:

        sami_data[col].tofile(data_out_path + 'raw-' + col, sep = ',')


np.array(sami_data[cols[0]].shape).tofile(data_out_path + 'raw-shape', sep = ',')

# In[ ]:



print('written! Exiting. ')

writing files...
written! Exiting. 
